In [10]:
"""
This script handles coordinate transformations using a custom DLL (`skt2_1507-1504_1.dll`) 
and a reference file (`HREF2018B_NN2000_EUREF89.bin`). It reads pairs of `.jpg` and `.sos` 
files, extracts the bounding coordinates from the `.sos` file, optionally transforms 
them from the Ålesund coordinate system to EUREF89 (using the DLL), and writes out a 
world file (`.jgw`) that allows GIS software to georeference the `.jpg`.

Important notes:
- The DLL (`skt2_1507-1504_1.dll`) and the reference file (`HREF2018B_NN2000_EUREF89.bin`)
  are expected to be for a Windows 32-bit environment that requires certain steps 
  for the transformation to succeed (e.g., correct paths, correct architecture).
- The script uses `ctypes` to interface with the DLL.
- The `.sos` file is assumed to contain lines specifying KoordSys, and coordinate pairs 
  under the `..NØ` marker.
"""

import os
import re
from PIL import Image
import ctypes


def parse_sos_file(sos_file_path):
    """
    Parse a `.sos` file to extract:
    1) The coordinate system (koordsys), if specified after '...KOORDSYS'
    2) The coordinate pairs after '..NØ'
    Returns:
        koordsys (int or None)
        coordinates (list of (float, float))
    """

    koordsys = None
    coordinates = []
    inside_coordinates = False

    # Open the .sos file in ISO8859-10 encoding.
    with open(sos_file_path, 'r', encoding='ISO8859-10') as f:
        for line in f:
            line = line.strip()
            # Find the line starting with ...KORDSYS to get the coordinate system number.
            if line.startswith('...KOORDSYS'):
                parts = line.split()
                if len(parts) >= 2:
                    koordsys = int(parts[1])
            # Find the line starting with ..NØ to get the coordinates on the subsequent lines.
            if line.startswith('..NØ'):
                inside_coordinates = True
                continue
            # If we are inside the coordinate section, parse the x and y coordinates as floats.
            if inside_coordinates:
                if re.match(r'^\d+\s+\d+$', line):
                    x, y = line.split()
                    coordinates.append((float(x), float(y)))
                else:
                    break

    return koordsys, coordinates


def initialize_transformation(dll_path, href_file):
    """
    Load the DLL at 'dll_path', then initialize the transformation with 'href_file'.
    Raises FileNotFoundError if either is missing.
    Raises RuntimeError if the DLL initialization fails (sErr != 0).
    Returns the DLL handle and references to _InitSkTrans and _GeoTrans functions.
    """

    # Check if the DLL file exists.
    if not os.path.exists(dll_path):
        raise FileNotFoundError("DLL file not found.")
    # Check if the HREF file exists.
    if not os.path.exists(href_file):
        raise FileNotFoundError("HREF file not found.")
    
    # Load the DLL with ctypes.
    trans_dll = ctypes.CDLL(dll_path)

    # Prepare references to the initialization function.
    _InitSkTrans = trans_dll._InitSkTrans
    _InitSkTrans.argtypes = [ctypes.c_char_p, ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short)]
    _InitSkTrans.restype = None

    # Encode HREF file path in utf-8 for passing to DLL.
    href = href_file.encode('utf-8')
    sLen = ctypes.c_short(len(href))
    sErr = ctypes.c_short(0)

    # Call the initialization function.
    _InitSkTrans(href, ctypes.byref(sLen), ctypes.byref(sErr))

    # Check if there was an error.
    if sErr.value != 0:
        raise RuntimeError(f"Failed to initialize transformation. Error Code: {sErr.value}")

    #Return references to the DLL file, the initialization function, and the main transformation function.
    return trans_dll, _InitSkTrans, trans_dll._GeoTrans


def transform_aalesund_to_euref(x, y, _GeoTrans, slSys1, slZone1, slSys2, slZone2):
    """
    Transform a single (x, y) from Ålesund's local system (Sys=4, Zone=1) to EUREF89 (Sys=7, Zone=32).
    _GeoTrans is the DLL's transformation function.
    The returned coordinates are (lat, lon) or (northing, easting), depending on the system.
    Raises RuntimeError if the transformation fails.
    """

    # Prepare input/outputs as ctypes doubles.
    x1 = ctypes.c_double(x)
    y1 = ctypes.c_double(y)
    z1 = ctypes.c_double(0.0)
    x2 = ctypes.c_double(0.0)
    y2 = ctypes.c_double(0.0)
    z2 = ctypes.c_double(0.0)
    slErr = ctypes.c_short(0)

    # Call the transformation function.
    _GeoTrans(
        ctypes.byref(slSys1), ctypes.byref(slZone1),
        ctypes.byref(x1), ctypes.byref(y1), ctypes.byref(z1),
        ctypes.byref(slSys2), ctypes.byref(slZone2),
        ctypes.byref(x2), ctypes.byref(y2), ctypes.byref(z2),
        ctypes.byref(slErr)
    )

    # Check for errors.
    if slErr.value != 0:
        raise RuntimeError(f"Transformation failed. Error Code: {slErr.value}")

    # Return the new coordinates containing northing and easting.
    return y2.value, x2.value


def create_world_file(image_path, x_min_trans, y_min_trans, x_max_trans, y_max_trans):
    """
    Create a .jgw (world file) based on an image's width/height and bounding coordinates.
    Args:
        image_path     : path to the .jpg image
        x_min_trans    : transformed min X (easting)
        y_min_trans    : transformed min Y (northing)
        x_max_trans    : transformed max X (easting)
        y_max_trans    : transformed max Y (northing)
    The .jgw file is named the same as the image but with .jgw extension.
    """

    # Open the image to get the width and height.
    img = Image.open(image_path)
    width, height = img.size

    # Pixel size in X range.
    pixel_size_x = (x_max_trans - x_min_trans) / width
    # Pixel size in Y range.
    pixel_size_y = (y_min_trans - y_max_trans) / height

    # Top-left corner in easting/northing.
    top_left_easting = x_min_trans
    top_left_northing = y_max_trans

    # Construct world-file path.
    world_file_path = os.path.splitext(image_path)[0] + ".jgw"
    # Write out the 6 lines of the world file.
    with open(world_file_path, 'w') as wf:
        wf.write(f"{pixel_size_x}\n")
        wf.write("0.0\n")
        wf.write("0.0\n")
        wf.write(f"{pixel_size_y}\n")
        wf.write(f"{top_left_easting}\n")
        wf.write(f"{top_left_northing}\n")

    print(f"World file created: {world_file_path}")


def process_folder(base_folder, dll_path, href_file):
    """
    Traverse all subfolders within 'base_folder'. 
    For each .jpg that has a matching .sos (same basename), parse the .sos for 2 coords.
    If KoordSys=110, transform from Ålesund local to EUREF. 
    Then create a .jgw (world file) so GIS software can georeference the .jpg.
    """

    # Walk through the base folder structure.
    for root, _, files in os.walk(base_folder):
        # Build dictionaries for .jpg and .sos by filename.
        jpg_files = {os.path.splitext(f)[0]: os.path.join(root, f) for f in files if f.endswith(".jpg")}
        sos_files = {os.path.splitext(f)[0]: os.path.join(root, f) for f in files if f.endswith(".sos")}

        # Find basenames that exist in both sets.
        matching_files = set(jpg_files.keys()) & set(sos_files.keys())
        for match in matching_files:
            jpg_path = jpg_files[match]
            sos_path = sos_files[match]

            # Parse the .sos file for coordinate system and coordinates.
            koordsys, coords = parse_sos_file(sos_path)
            if len(coords) != 2:
                print(f"Skipping {sos_path} - Invalid coordinate pairs")
                continue

            # Unpack the two coordinates pairs (x_min, y_min), (x_max, y_max)
            (x_min, y_min), (x_max, y_max) = coords
            x_min, x_max = sorted([x_min, x_max])
            y_min, y_max = sorted([y_min, y_max])

            # If the coordinate system is 110, do custom transformation, else use the coordinates as is.
            if koordsys == 110:
                try:
                    # Initialize the transformation library.
                    trans_dll, _InitSkTrans, _GeoTrans = initialize_transformation(dll_path, href_file)

                    # Set up the system codes for the DLL.
                    slSys1 = ctypes.c_short(4) # Source system.
                    slZone1 = ctypes.c_short(1)
                    slSys2 = ctypes.c_short(7) # Target system (EUREF).
                    slZone2 = ctypes.c_short(32)

                    # Helper function to transform a single .sos/.jpg pair.
                    def transform(x, y):
                        return transform_aalesund_to_euref(x, y, _GeoTrans, slSys1, slZone1, slSys2, slZone2)

                    # Transform both corners.
                    easting_min, northing_min = transform(x_min, y_min)
                    easting_max, northing_max = transform(x_max, y_max)
                except RuntimeError as e:
                    print(f"Error transforming {sos_path}: {e}")
                    continue
            else:
                # If the coordinate system is not 110, just use the coordinates as is.
                easting_min, northing_min = x_min, y_min
                easting_max, northing_max = x_max, y_max

            # Create the new .jgw world file.
            create_world_file(jpg_path, easting_min, northing_min, easting_max, northing_max)


if __name__ == "__main__":
    base_folder = "../dataset"
    dll_path = r"skt2_1507-1504_1.dll"
    href_file = r"HREF2018B_NN2000_EUREF89.bin"

    process_folder(base_folder, dll_path, href_file)


World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200b.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200f.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200d.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200h.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200g.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200e.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504200\200i.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201k.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201b.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201c.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504201\201e.jgw
World fi

c:\Users\SivertEspelandHusebø\python32\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (171714688 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


World file created: ../dataset\dataset\aalesund\FOKUS\1504250\250.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504250\250c.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504251\251.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504252\252.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504256\256.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504261\261a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504261\261.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504262A\262a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263j.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263i.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263e.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263a.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263c.jgw
World file created: ../dataset\dataset\aalesund\FOKUS\1504263\263d.jgw
World file